In [0]:
# Install Pyomo 
%pip install pyomo

In [0]:
dbutils.library.restartPython()

In [0]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import mlflow
import os

In [0]:
%sh bash install_cbc.sh

In [0]:
suppliers = ['S1', 'S2']
factories = ['F1', 'F2']
customers = ['C1', 'C2']

# Sample demands
demand = {'C1': 40, 'C2': 50}

# Factory capacity (can be disrupted)
factory_capacity = {'F1': 60, 'F2': 70}
extra_capacity = {'F1': 20, 'F2': 30}  # dynamic surge capacity

# Cost per unit
ship_cost = {
    ('S1', 'F1'): 2, ('S1', 'F2'): 3,
    ('S2', 'F1'): 4, ('S2', 'F2'): 2,
    ('F1', 'C1'): 1, ('F1', 'C2'): 2,
    ('F2', 'C1'): 3, ('F2', 'C2'): 1,
}

# Cost multipliers
extra_cost = {'F1': 5, 'F2': 5}
unmet_penalty = 100

In [0]:
def build_model(factory_caps, extra_caps):
    model = pyo.ConcreteModel()  # Create a concrete optimization model
    model.S = pyo.Set(initialize=suppliers)  # Define a set of suppliers
    model.F = pyo.Set(initialize=factories)  # Define a set of factories
    model.C = pyo.Set(initialize=customers)  # Define a set of customers
    
    model.demand = pyo.Param(model.C, initialize=demand)  # Parameter for customer demand
    model.factory_cap = pyo.Param(model.F, initialize=factory_caps, mutable=True)  # Parameter for factory capacity
    model.extra_cap = pyo.Param(model.F, initialize=extra_caps, mutable=True)  # Parameter for extra capacity
    model.ship_cost = pyo.Param(model.S * model.F | model.F * model.C, initialize=ship_cost)  # Shipping cost parameter
    model.extra_cost = pyo.Param(model.F, initialize=extra_cost)  # Extra cost parameter for additional capacity
    
    # Vars
    model.x_sf = pyo.Var(model.S, model.F, domain=pyo.NonNegativeReals)  # Decision variable for shipments from suppliers to factories
    model.x_fc = pyo.Var(model.F, model.C, domain=pyo.NonNegativeReals)  # Decision variable for shipments from factories to customers
    model.extra_used = pyo.Var(model.F, domain=pyo.NonNegativeReals)  # Decision variable for extra capacity used
    model.unmet = pyo.Var(model.C, domain=pyo.NonNegativeReals)  # Decision variable for unmet demand
    
    # Flow into factories must equal flow out
    def flow_balance_rule(m, f):
        return sum(m.x_sf[s, f] for s in m.S) == sum(m.x_fc[f, c] for c in m.C)  # Flow balance constraint for factories
    model.flow_balance = pyo.Constraint(model.F, rule=flow_balance_rule)  # Apply flow balance constraint

    # Factory capacity constraint
    def cap_rule(m, f):
        return sum(m.x_fc[f, c] for c in m.C) <= m.factory_cap[f] + m.extra_used[f]  # Capacity constraint for factories
    model.cap = pyo.Constraint(model.F, rule=cap_rule)  # Apply capacity constraint

    # Extra capacity limits
    def extra_limit(m, f):
        return m.extra_used[f] <= m.extra_cap[f]  # Limit on extra capacity used
    model.extra_limit = pyo.Constraint(model.F, rule=extra_limit)  # Apply extra capacity limit

    # Demand fulfillment
    def demand_rule(m, c):
        return sum(m.x_fc[f, c] for f in m.F) + m.unmet[c] == m.demand[c]  # Demand fulfillment constraint
    model.demand_fill = pyo.Constraint(model.C, rule=demand_rule)  # Apply demand fulfillment constraint
    
    # Objective: total cost
    model.obj = pyo.Objective(
        expr = sum(model.ship_cost[s, f] * model.x_sf[s, f] for s in model.S for f in model.F) +
               sum(model.ship_cost[f, c] * model.x_fc[f, c] for f in model.F for c in model.C) +
               sum(model.extra_cost[f] * model.extra_used[f] for f in model.F) +
               sum(unmet_penalty * model.unmet[c] for c in model.C),  # Objective function to minimize total cost
        sense=pyo.minimize  # Set the objective to minimize
    )

    return model  # Return the constructed model

In [0]:
experiment_name = "/SupplyChainStressTest"